In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

import pandas as pd
import numpy as np

# 1. Regression - Let's predict house prices

## The Ames Housing dataset

http://jse.amstat.org/v19n3/decock.pdf


"This paper presents a data set describing the sale of individual residential property in Ames, Iowa
from 2006 to 2010. The data set contains 2930 observations and a large number of explanatory
variables (23 nominal, 23 ordinal, 14 discrete, and 20 continuous) involved in assessing home
values. I will discuss my previous use of the Boston Housing Data Set and I will suggest
methods for incorporating this new data set as a final project in an undergraduate regression
course."


80 total features
2930 examples



In [ ]:
df_dataset = pd.read_csv('http://jse.amstat.org/v19n3/decock/AmesHousing.txt', sep='\t')
df_dataset.head()

In [ ]:
df_dataset.shape

In [ ]:
df_dataset.columns

To make things simple, lets limit the use of our features to start

In [ ]:
columns = ['Overall Qual', 'Overall Cond', 'Gr Liv Area', 'Central Air', 'Total Bsmt SF', 'SalePrice']
df = df_dataset[columns]
df.head()

In [ ]:
df.dtypes


We need to deal with string data in some way. Notice Central air and building type are  object types

In [ ]:
df['Central Air'].value_counts()

Since there are two options for this feature, we can encode it as 0 and 1

In [ ]:
df['Central Air'] = df['Central Air'].map({'N': 0, 'Y': 1})

In [ ]:
df.head()

## Visualizing Data

Let's take a look at our data. Try to experiment with mulitple different libraries and see which one you like. Seaborn is neat:


In [ ]:

sns.pairplot(
    df,
    corner=True,
    height=2
)


## Linear Model

We'll start with just two columns in the data set.  `Overall Qual` and `SalePrice`.  Use Scikit-Learn's [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) to make a simple model that predicts income based on age.  

In [ ]:
from sklearn.linear_model import LinearRegression

X = df[['Overall Qual']]
y= df['SalePrice']

X,y = X.values, y.values

In [ ]:
price_est = LinearRegression()

price_est.fit(X,y)

In [ ]:
price_est.score(X,y)

In [ ]:
price_est.coef_

In [ ]:
price_est.intercept_

Lets plot and see how our estimator worked:

In [ ]:
f = sns.scatterplot(x=X.flatten(),y=y)
plt.plot(X, price_est.predict(X), color='black', lw=2)

f.set_xlabel('Overal Quality')
f.set_ylabel('Sale Price ($)')

In [ ]:
price_est.score(X,y)

Here the low $R^2$ score indicates underfitting.  Our model isn't very good, which shouldn't suprise us since we've ignored most of our data. 

## Incorporate all features, and setting aside a proper testing set

In this past section, we skipped setting apart a test set for validating our model. Let's do that before we get started

In [ ]:
df.columns

In [ ]:
from sklearn.model_selection import train_test_split


target = 'SalePrice'
features = df.columns[df.columns != target]

X = df[features]
y= df[target]


X,y = X.values, y.values # Convert to numpy arrays

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
print(f'x train shape: {X_train.shape}')
print(f'x test shape: {X_test.shape}')

In [ ]:

# df[df.isna().any(axis=1)]
# df = df.dropna()

In [ ]:
price_est = LinearRegression()
price_est.fit(X_train,y_train)

In [ ]:
price_est.score(X_test, y_test)

Since our model uses multiple variables, we can't visualize the regression using a 2D plot. Instead, we can plot the residuals  (the differences or vertical distances between the actual and predicted values) versus the predicted values to diagnose our regression model. 

In [ ]:
y_train_pred = price_est.predict(X_train)
y_test_pred = price_est.predict(X_test)

In [ ]:
y_train_pred = price_est.predict(X_train)
y_test_pred = price_est.predict(X_test)

x_max = np.max([np.max(y_train_pred), np.max(y_test_pred)])
x_min = np.min([np.min(y_train_pred), np.min(y_test_pred)])

fig, (ax1, ax2) = plt.subplots( 1, 2, figsize=(7, 3), sharey=True)

ax1.scatter(
    y_test_pred, y_test_pred - y_test,
    c='limegreen', marker='s',
    edgecolor='white',
    label='Test data')
ax2.scatter(
    y_train_pred, y_train_pred - y_train,
    c='steelblue', marker='o', edgecolor='white',
    label='Training data')
ax1.set_ylabel('Residuals')

for ax in (ax1, ax2):
    ax.set_xlabel('Predicted values')
    ax.legend(loc='upper left')
    ax.hlines(y=0, xmin=x_min-100, xmax=x_max+100,\
    color='black', lw=2)

plt.tight_layout()
plt.show()

In [ ]:
 from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

In [ ]:
print(f'MAE train: {mae_train:.2f}')
print(f'MAE test: {mae_test:.2f}')

In [ ]:
print('R2 train', price_est.score(X_train, y_train))
print('R2 test', price_est.score(X_test, y_test))


## Polynomial Features

Linear models cannot detect interactions between features. One way around this limitation is to create new features that encode the interactions we're interested in.  For example, we can use the values given by the product of each pair (or tuple) of features.  This is exactly what Scikit-Learn's [`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) transformer does.

## Nonlinear Regression -> Random forest

We can also deal with nonlinear relationships by moving away from linear models. We can try out random forest regression, which is an ensemble of multiple
decision trees, in contrast to the global linear regression model that we discussed previously.

Let's also add some extra features in our working dataset

In [ ]:
columns = ['Overall Qual', 'Overall Cond', 'Gr Liv Area', 'Central Air', 'Total Bsmt SF', 'House Style',  'SalePrice']
df = df_dataset[columns]
df.head()

df['Central Air'] = df['Central Air'].map({'N': 0, 'Y': 1})
df.head()


# Clean up any Nans...
df[df.isna().any(axis=1)]
print('Found this many nans:', df[df.isna().any(axis=1)].shape)
df = df.dropna()


In [ ]:

target = 'SalePrice'
features = df.columns[df.columns != target]

X = df[features]
y= df[target]


X,y = X.values, y.values # Convert to numpy arrays

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

### One hot encoding

In [ ]:
df['House Style'].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_transformer = OneHotEncoder(sparse=False)
res  = one_hot_transformer.fit_transform(df[['House Style']].values)

In [ ]:
res

In [ ]:
one_hot_transformer.n_features_in_

In [ ]:
one_hot_transformer.categories_

In [ ]:
one_hot_transformer.get_feature_names_out()

### Pipelines

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [ ]:
df.columns

In [ ]:
transformer_pipeline = Pipeline(
    [
        ("onehotencoder", ColumnTransformer(
            [
                ("onehotencoder", OneHotEncoder(sparse=False), [5]),
                ( 'nada', 'passthrough', [0,1,2,3,4])
            ]
        ))
    ]
)


X_transformed = transformer_pipeline.fit_transform(X)

In [ ]:
X_transformed[0].shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest = RandomForestRegressor(
    n_estimators=1000,
     criterion='squared_error',
     random_state=22,
     n_jobs=-1)

In [ ]:
pipeline = Pipeline(
    [
        ("onehotencoder", ColumnTransformer(
            [
                ("onehotencoder", OneHotEncoder(sparse=False), [5]),
                ( 'nada', 'passthrough', [0,1,2,3,4])
            ]
        )),
        (
            "random_forest", forest
        )
    ]
)


pipeline.fit(X_train, y_train)

In [ ]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

x_max = np.max([np.max(y_train_pred), np.max(y_test_pred)])
x_min = np.min([np.min(y_train_pred), np.min(y_test_pred)])

fig, (ax1, ax2) = plt.subplots( 1, 2, figsize=(7, 3), sharey=True)

ax1.scatter(
    y_test_pred, y_test_pred - y_test,
    c='limegreen', marker='s',
    edgecolor='white',
    label='Test data')
ax2.scatter(
    y_train_pred, y_train_pred - y_train,
    c='steelblue', marker='o', edgecolor='white',
    label='Training data')
ax1.set_ylabel('Residuals')

for ax in (ax1, ax2):
    ax.set_xlabel('Predicted values')
    ax.legend(loc='upper left')
    ax.hlines(y=0, xmin=x_min-100, xmax=x_max+100,\
    color='black', lw=2)

plt.tight_layout()
plt.show()

In [ ]:
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'MAE train: {mae_train:.2f}')
print(f'MAE test: {mae_test:.2f}')

In [ ]:
print('R2 train', pipeline.score(X_train, y_train))
print('R2 test', pipeline.score(X_test, y_test))


### Saving your model

In [ ]:
import pickle

In [ ]:
with open('HousePricePredictor.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [ ]:
with open('HousePricePredictor.pkl', 'rb') as f:
    reloaded_pipeline = pickle.load(f)

Test to make sure it works

In [ ]:
reloaded_pipeline.predict([X_test[45]])

## Alternative way of one hot encoding

Make use of pandas `get_dummys` function.

Think about the difference between approaches. When might this be useful?

In [ ]:
columns = ['Overall Qual', 'Overall Cond', 'Gr Liv Area', 'Central Air', 'Total Bsmt SF', 'House Style',  'SalePrice']
df = df_dataset[columns]
df.head()

df['Central Air'] = df['Central Air'].map({'N': 0, 'Y': 1})
df.head()


# Clean up any Nans...
df[df.isna().any(axis=1)]
print('Found this many nans:', df[df.isna().any(axis=1)].shape)
df = df.dropna()
df.head()

In [ ]:
pd.get_dummies(df['House Style'])

In [ ]:
from sklearn.datasets import load_diabetes
diabetes_data = load_diabetes()